# Net Position Analysis: Three-System Comparison

This notebook compares net position measurements from three independent systems:
1. **FFT Signal Processing** - High-precision frequency domain analysis
2. **Sonar Image Analysis** - Computer vision-based detection from multibeam sonar
3. **DVL Navigation** - Doppler velocity log-based positioning (reference)

All measurements are synchronized by timestamp and compared in:
- Distance measurements
- Pitch angle estimates
- XY position coordinates (net-relative frame)

## 1. Import Libraries and Configuration

In [1]:
from pathlib import Path
import numpy as np
from utils.net_analysis import (
    load_sonar_analysis_results,
    load_navigation_dataset,
    load_fft_dataset,
    apply_time_range_filter,
    apply_smoothing_to_all_systems,
    print_data_summaries,
    print_sample_data,
    synchronize_sonar_and_dvl,
    summarize_distance_alignment,
    prepare_three_system_comparison,
    ensure_xy_columns,
    generate_and_save_xy_plots,
    print_xy_position_statistics,
    compute_distance_pitch_statistics,
    print_distance_pitch_statistics,
)

# Import config for verification
from utils.config import IMAGE_PROCESSING_CONFIG, TRACKING_CONFIG

# CONFIG VERIFICATION - Print exact values to ensure consistency
print("=== IMAGE_PROCESSING_CONFIG VALUES ===")
for key, value in IMAGE_PROCESSING_CONFIG.items():
    print(f"{key}: {value}")

print("\n=== TRACKING_CONFIG VALUES ===")
for key, value in TRACKING_CONFIG.items():
    print(f"{key}: {value}")

print("\nUtility modules ready!")

 CONFIG VALIDATION WARNING:
   Missing TRACKING_CONFIG keys: {'corridor_widen', 'use_corridor_splitting', 'corridor_both_directions'}
=== IMAGE_PROCESSING_CONFIG VALUES ===
binary_threshold: 128
use_advanced_momentum_merging: True
adaptive_angle_steps: 20
adaptive_base_radius: 3
adaptive_max_elongation: 1.0
momentum_boost: 10.0
adaptive_linearity_threshold: 0.75
downscale_factor: 2
top_k_bins: 8
min_coverage_percent: 0.3
gaussian_sigma: 5.0
basic_gaussian_kernel_size: 3
basic_gaussian_sigma: 1.0
basic_use_dilation: True
basic_dilation_kernel_size: 3
basic_dilation_iterations: 3
morph_close_kernel: 0
edge_dilation_iterations: 0
min_contour_area: 200
aoi_boost_factor: 10.0
max_distance_change_pixels: 20

=== TRACKING_CONFIG VALUES ===
use_elliptical_aoi: True
ellipse_expansion_factor: 0.5
aoi_boost_factor: 10.0
score_area_weight: 1.0
score_linearity_weight: 2.0
score_aspect_ratio_weight: 1.5
aoi_center_x_percent: 50
aoi_center_y_percent: 60
aoi_width_percent: 60
aoi_height_percent: 70
ce

## 2. Configuration and Setup

In [2]:
# Target bag to analyze
TARGET_BAG = "2024-08-20_17-02-00"  # Change this to your desired bag ID

# Time range selection (None = use all data)
TIME_START = 5    # Start time: seconds from first timestamp OR ISO datetime string
TIME_END = 35     # End time: seconds from first timestamp OR ISO datetime string
# Examples:
#   TIME_START = None, TIME_END = None              # All data
#   TIME_START = 0, TIME_END = 60                   # First 60 seconds
#   TIME_START = 30, TIME_END = 90                  # 30-90 seconds from start
#   TIME_START = 120, TIME_END = None               # From 120 seconds to end
#   TIME_START = "2024-08-20T17:02:30", TIME_END = "2024-08-20T17:03:30"  # Absolute times

# Smoothing configuration (None or 1.0 = no smoothing)
SMOOTHING_ALPHA = 0.1  # Exponential smoothing factor (0-1)
# Examples:
#   SMOOTHING_ALPHA = None   # No smoothing (raw data)
#   SMOOTHING_ALPHA = 0.1    # Heavy smoothing (90% history, 10% current)
#   SMOOTHING_ALPHA = 0.3    # Moderate smoothing (70% history, 30% current)
#   SMOOTHING_ALPHA = 0.5    # Balanced smoothing (50% history, 50% current)
#   SMOOTHING_ALPHA = 0.7    # Light smoothing (30% history, 70% current)
#   SMOOTHING_ALPHA = 0.9    # Very light smoothing (10% history, 90% current)

# Data source paths
FFT_ROOT = Path("/Volumes/LaCie/SOLAQUA/relative_fft_pose")
PLOTS_DIR = Path("/Volumes/LaCie/SOLAQUA/exports/plots")
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Analyzing: {TARGET_BAG}")
if TIME_START is not None or TIME_END is not None:
    if isinstance(TIME_START, (int, float)) or isinstance(TIME_END, (int, float)):
        print(f"Time range: {TIME_START or 'start'}s to {TIME_END or 'end'}s (relative)")
    else:
        print(f"Time range: {TIME_START or 'start'} to {TIME_END or 'end'} (absolute)")
else:
    print(f"Time range: All data")

if SMOOTHING_ALPHA is not None and SMOOTHING_ALPHA < 1.0:
    print(f"Smoothing: Enabled (α={SMOOTHING_ALPHA:.2f})")
else:
    print(f"Smoothing: Disabled (raw data)")

print(f"Plots: {PLOTS_DIR}")
print(f"\nNOTE: To export data for this bag, use:")
print(f"python scripts/solaqua_export.py --data-dir /Volumes/LaCie/SOLAQUA/raw_data \\")
print(f"  --exports-dir /Volumes/LaCie/SOLAQUA/exports --bag-stem {TARGET_BAG} --all")

Analyzing: 2024-08-20_17-02-00
Time range: 5s to 35s (relative)
Smoothing: Enabled (α=0.10)
Plots: /Volumes/LaCie/SOLAQUA/exports/plots

NOTE: To export data for this bag, use:
python scripts/solaqua_export.py --data-dir /Volumes/LaCie/SOLAQUA/raw_data \
  --exports-dir /Volumes/LaCie/SOLAQUA/exports --bag-stem 2024-08-20_17-02-00 --all


In [3]:
# Load data from all three systems
print("=== LOADING DATA ===\n")

df_sonar, sonar_meta = load_sonar_analysis_results(TARGET_BAG)
df_nav, nav_meta = load_navigation_dataset(TARGET_BAG)
df_fft, fft_meta = load_fft_dataset(TARGET_BAG, fft_root=FFT_ROOT)

# Apply time range selection if specified
df_sonar, df_nav, df_fft = apply_time_range_filter(df_sonar, df_nav, df_fft, TIME_START, TIME_END)

# Apply smoothing if specified (before any analysis)
if SMOOTHING_ALPHA is not None and SMOOTHING_ALPHA < 1.0:
    df_sonar, df_nav, df_fft = apply_smoothing_to_all_systems(df_sonar, df_nav, df_fft, SMOOTHING_ALPHA)

# Display data sources
print("Data Sources:")
for label, meta in {"Sonar": sonar_meta, "DVL": nav_meta, "FFT": fft_meta}.items():
    path = meta.get('resolved_path', 'Not found')
    rows = meta.get('rows', 0)
    status = "✓" if rows > 0 else "✗"
    print(f"  {status} {label}: {rows} records")
    if rows > 0:
        print(f"      {path}")

=== LOADING DATA ===

Applying time range filter: 5 to 35

  Sonar: 944 → 469 records
  DVL: 497 → 269 records
  FFT: 410 → 235 records


=== APPLYING EXPONENTIAL SMOOTHING (α=0.10) ===
  Lower α = more smoothing (e.g., 0.1 = heavy smoothing)
  Higher α = less smoothing (e.g., 0.9 = light smoothing)

  Sonar distance: 0.956 → 0.960 m (mean)
  DVL distance: 1.007 → 1.009 m (mean)
  FFT distance: 0.877 → 0.884 m (mean)
✓ Smoothing applied

Data Sources:
  ✓ Sonar: 944 records
      /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_17-02-00_analysis.csv
  ✓ DVL: 497 records
      /Volumes/LaCie/SOLAQUA/exports/by_bag/navigation_plane_approximation__2024-08-20_17-02-00_data.csv
  ✓ FFT: 410 records
      /Volumes/LaCie/SOLAQUA/relative_fft_pose/2024-08-20_17-02-00_relative_pose_fft.csv


In [4]:
# Summaries for each dataset
print_data_summaries(df_sonar, df_nav, df_fft)


=== DATA SUMMARIES ===

Sonar:
  rows: 469
  detections: 469
  detection_rate: 100.000
  distance_range_m: 0.913 to 1.141
  angle_range_deg: 169.859 to 182.847

DVL:
  rows: 269
  distance_range_m: 0.982 to 1.060
  distance_mean_m: 1.009
  distance_std_m: 0.015
  pitch_range_deg: -8.549 to 1.796

FFT:
  rows: 235
  distance_range_m: 0.616 to 1.085
  distance_mean_m: 0.884
  distance_std_m: 0.083
  pitch_range_deg: 0.162 to 11.167



In [5]:
# Display sample data
print_sample_data(df_sonar, df_nav, df_fft)

Sonar Analysis Sample:
                             timestamp  distance_meters  angle_degrees  \
79 2024-08-20 15:02:08.369414568+00:00         1.127285     169.859259   
80 2024-08-20 15:02:08.426076412+00:00         1.127556     169.861245   
81 2024-08-20 15:02:08.497674465+00:00         1.127684     169.880001   
82 2024-08-20 15:02:08.550859928+00:00         1.127791     169.915904   
83 2024-08-20 15:02:08.617468834+00:00         1.128444     169.956016   

    detection_success  
79               True  
80               True  
81               True  
82               True  
83               True  

DVL Navigation Sample:
                             timestamp  NetDistance  NetPitch
44 2024-08-20 15:02:10.413454771+00:00     1.060000 -0.070490
45 2024-08-20 15:02:10.526475668+00:00     1.058000 -0.068646
46 2024-08-20 15:02:10.623238802+00:00     1.056200 -0.069682
47 2024-08-20 15:02:10.724297762+00:00     1.054580 -0.067820
48 2024-08-20 15:02:10.846749783+00:00     1.052122 -0

## 4. Two-System Synchronization (Sonar + DVL)

First, compare sonar and DVL systems independently.

In [6]:
# Synchronize sonar and DVL measurements
print("=== TWO-SYSTEM SYNCHRONIZATION (Sonar + DVL) ===\n")

sync_df = synchronize_sonar_and_dvl(df_sonar, df_nav, tolerance_seconds=1.0)
sync_summary = summarize_distance_alignment(sync_df)

print(f"Aligned pairs: {sync_summary['pairs']}")
print(f"Valid distance pairs: {sync_summary['valid_pairs']}")

if sync_summary["difference_stats"]:
    diff = sync_summary["difference_stats"]
    print(f"\nDistance Comparison (Sonar - DVL):")
    print(f"  Mean: {diff['mean']:.3f} m")
    print(f"  Std: {diff['std']:.3f} m")
    print(f"  Range: {diff['min']:.3f} to {diff['max']:.3f} m")
    print(f"  95th percentile: {diff['p95']:.3f} m")
    if 'correlation' in diff:
        print(f"  Correlation: {diff['correlation']:.3f}")
else:
    print("No overlapping distance measurements within tolerance.")

# Display sample synchronized data
if not sync_df.empty:
    print(f"\nSynchronized Data Sample:")
    display_cols = ['timestamp', 'sonar_distance_m', 'dvl_distance_m', 'sonar_angle_deg', 'dvl_pitch_deg', 'time_diff_s']
    display_cols = [c for c in display_cols if c in sync_df.columns]
    print(sync_df[display_cols].head(10))

=== TWO-SYSTEM SYNCHRONIZATION (Sonar + DVL) ===

Aligned pairs: 452
Valid distance pairs: 452

Distance Comparison (Sonar - DVL):
  Mean: -0.058 m
  Std: 0.031 m
  Range: -0.100 to 0.079 m
  95th percentile: 0.014 m
  Correlation: 0.842

Synchronized Data Sample:
                            timestamp  sonar_distance_m  dvl_distance_m  \
0 2024-08-20 15:02:09.455889225+00:00          1.138591            1.06   
1 2024-08-20 15:02:09.517638445+00:00          1.137448            1.06   
2 2024-08-20 15:02:09.576431036+00:00          1.136009            1.06   
3 2024-08-20 15:02:09.640974283+00:00          1.133597            1.06   
4 2024-08-20 15:02:09.706049681+00:00          1.131262            1.06   
5 2024-08-20 15:02:09.776943207+00:00          1.128941            1.06   
6 2024-08-20 15:02:09.844408751+00:00          1.125734            1.06   
7 2024-08-20 15:02:09.913848400+00:00          1.122789            1.06   
8 2024-08-20 15:02:09.964217663+00:00          1.119506     

/Users/eirikvarnes/code/SOLAQUA/utils/net_analysis.py:210: UserWarning: no explicit representation of timezones available for np.datetime64
  sonar_ts = np.datetime64(sonar_row.timestamp)


## 5. Three-System Comparison

Synchronize and compare all three systems (FFT + Sonar + DVL).

In [7]:
# Perform multi-system comparison (works with 2 or 3 systems)
print("\n=== MULTI-SYSTEM COMPARISON ===\n")

# Determine which systems are available
available_systems = []
if df_sonar is not None and not df_sonar.empty:
    available_systems.append("Sonar")
if df_nav is not None and not df_nav.empty:
    available_systems.append("DVL")
if df_fft is not None and not df_fft.empty:
    available_systems.append("FFT")

print(f"Available systems: {', '.join(available_systems)}")

if len(available_systems) < 2:
    print("✗ Need at least 2 systems for comparison")
    three_sync_df = None
    figs = {}
    visualizer = None
else:
    # Run multi-system comparison
    system_count = len(available_systems)
    print(f"Running {system_count}-system comparison...")
    
    three_sync_df, figs, visualizer = prepare_three_system_comparison(
        TARGET_BAG,
        df_sonar,
        df_nav,
        df_fft,
        tolerance_seconds=0.5
    )
    
    if three_sync_df is None or three_sync_df.empty:
        print("✗ No overlapping timestamps across available systems")
        three_sync_df = None
    else:
        print(f"✓ Synchronized {len(three_sync_df)} records across {system_count} systems\n")
        
        # Print available columns
        print("Available columns:")
        distance_cols = [c for c in three_sync_df.columns if 'distance' in c.lower()]
        pitch_cols = [c for c in three_sync_df.columns if 'pitch' in c.lower() or 'angle' in c.lower()]
        print(f"  Distance: {distance_cols}")
        print(f"  Pitch: {pitch_cols}\n")
        
        # Ensure XY columns exist
        three_sync_df = ensure_xy_columns(three_sync_df)


=== MULTI-SYSTEM COMPARISON ===

Available systems: Sonar, DVL, FFT
Running 3-system comparison...
✓ Synchronized 973 records across 3 systems

Available columns:
  Distance: ['fft_distance_m', 'sonar_distance_m', 'nav_distance_m']
  Pitch: ['fft_pitch_deg', 'sonar_pitch_deg', 'nav_pitch_deg']

✓ XY coordinates already exist: ['fft_x_m', 'fft_y_m', 'sonar_x_m', 'sonar_y_m', 'nav_x_m', 'nav_y_m']
   Renamed columns: ['fft_x_m', 'fft_y_m', 'sonar_x_m', 'sonar_y_m', 'nav_x_m', 'nav_y_m'] → ['fft_x', 'fft_y', 'sonar_x', 'sonar_y', 'dvl_x', 'dvl_y']
✓ Synchronized 973 records across 3 systems

Available columns:
  Distance: ['fft_distance_m', 'sonar_distance_m', 'nav_distance_m']
  Pitch: ['fft_pitch_deg', 'sonar_pitch_deg', 'nav_pitch_deg']

✓ XY coordinates already exist: ['fft_x_m', 'fft_y_m', 'sonar_x_m', 'sonar_y_m', 'nav_x_m', 'nav_y_m']
   Renamed columns: ['fft_x_m', 'fft_y_m', 'sonar_x_m', 'sonar_y_m', 'nav_x_m', 'nav_y_m'] → ['fft_x', 'fft_y', 'sonar_x', 'sonar_y', 'dvl_x', 'dvl_

## 6. Distance and Pitch Comparison Plots

In [8]:
# Display comparison plots
if three_sync_df is not None and not three_sync_df.empty and figs:
    print("=== DISPLAYING COMPARISON PLOTS ===\n")
    
    # Display all generated plots
    for name, fig in figs.items():
        if fig is not None:
            print(f"Displaying {name.replace('_', ' ').title()}...")
            try:
                fig.show()
                
                # Save plots
                save_path = PLOTS_DIR / f"{TARGET_BAG}_{name}.html"
                fig.write_html(str(save_path))
                print(f"  ✓ Saved: {save_path.name}\n")
            except Exception as e:
                print(f"  ✗ Warning: Could not display/save - {e}\n")
    
    print("✓ All plots displayed and saved")
else:
    print("✗ No synchronized data or plots available")
    print("  This usually means:")
    print("  - Timestamps don't overlap between systems")
    print("  - Data quality issues")
    print("  - Missing required columns in datasets")

=== DISPLAYING COMPARISON PLOTS ===

Displaying Distance Comparison...


  ✓ Saved: 2024-08-20_17-02-00_distance_comparison.html

Displaying Pitch Comparison...


  ✓ Saved: 2024-08-20_17-02-00_pitch_comparison.html

✓ All plots displayed and saved


## 7. Net-Relative XY Position Plots

Visualize the robot's trajectory relative to the net plane in XY coordinates from all systems.

In [9]:
# Display XY position plots
if three_sync_df is not None and not three_sync_df.empty and visualizer is not None:
    # Configure lateral speed (robot moving along net)
    LATERAL_SPEED_M_S = 0.2  # meters per second (adjust based on actual speed)
    
    # Generate and save all XY plots
    xy_plot_results = generate_and_save_xy_plots(
        three_sync_df, 
        visualizer, 
        TARGET_BAG, 
        PLOTS_DIR, 
        lateral_speed_m_s=LATERAL_SPEED_M_S
    )
else:
    print("✗ No synchronized data or visualizer available for XY plots")

=== DISPLAYING XY POSITION PLOTS ===

Systems with XY coordinates: Sonar, DVL, FFT

Generating 3D XY trajectory plot...
  Lateral speed: 0.2 m/s


✓ Saved: 2024-08-20_17-02-00_xy_trajectories_3d.html


Generating XY component comparison plots...

--- Perpendicular Distance to Net Over Time ---


✓ X position comparison saved: 2024-08-20_17-02-00_x_position_comparison.html

--- Lateral Position Along Net Over Time ---


✓ Y position comparison saved: 2024-08-20_17-02-00_y_position_comparison.html

✓ XY position plots complete


## 8. Position Statistics and Analysis

In [10]:
# Display detailed position statistics
if three_sync_df is not None and not three_sync_df.empty:
    # XY Position Summary
    print_xy_position_statistics(three_sync_df)
    
    # Distance and Pitch Statistics
    distance_pitch_stats = compute_distance_pitch_statistics(three_sync_df)
    print_distance_pitch_statistics(distance_pitch_stats)
    
    # Additional detailed analysis if visualizer available
    if visualizer is not None:
        try:
            print("\n=== DETAILED POSITION ANALYSIS ===\n")
            visualizer.print_position_summary(three_sync_df)
        except Exception as e:
            print(f"Could not print detailed analysis: {e}")
else:
    print("✗ No synchronized data available for position analysis")

=== POSITION STATISTICS ===

FFT:
  Data points: 960
  X position: 0.870 ± 0.109 m
  Y position: 0.066 ± 0.086 m
  Distance from origin: 0.877 ± 0.107 m

Sonar:
  Data points: 958
  X position: 0.956 ± 0.054 m
  Y position: -0.081 ± 0.046 m
  Distance from origin: 0.960 ± 0.056 m

DVL:
  Data points: 935
  X position: 1.009 ± 0.017 m
  Y position: -0.042 ± 0.036 m
  Distance from origin: 1.011 ± 0.017 m

=== DISTANCE & PITCH STATISTICS ===

Distance Measurements:
  FFT:
    Mean: 0.881 m
    Std: 0.081 m
    Min: 0.616 m
    Max: 1.085 m
    Range: 0.470 m
  Sonar:
    Mean: 0.960 m
    Std: 0.056 m
    Min: 0.913 m
    Max: 1.141 m
    Range: 0.228 m
  DVL:
    Mean: 1.011 m
    Std: 0.017 m
    Min: 0.982 m
    Max: 1.060 m
    Range: 0.078 m

Pitch/Angle Measurements:
  FFT:
    Mean: 4.4°
    Std: 2.7°
    Min: 0.2°
    Max: 11.2°
    Range: 11.0°
  Sonar:
    Mean: -4.8°
    Std: 2.6°
    Min: -10.1°
    Max: 2.8°
    Range: 13.0°
  DVL:
    Mean: -2.4°
    Std: 2.1°
    Min: -8.5

## 9. Summary

In [11]:
print("\n" + "="*70)
print("ANALYSIS SUMMARY")
print("="*70)

print(f"\nTarget Bag: {TARGET_BAG}")

print(f"\nData Availability:")
for label, meta in {"Sonar": sonar_meta, "DVL": nav_meta, "FFT": fft_meta}.items():
    rows = meta.get('rows', 0)
    status = "✓" if rows > 0 else "✗"
    print(f"  {status} {label}: {rows} records")

print(f"\nSynchronization Results:")
print(f"  Two-system (Sonar+DVL): {len(sync_df)} records")
if three_sync_df is not None:
    print(f"  Multi-system: {len(three_sync_df)} records")
else:
    print(f"  Multi-system: N/A")

if three_sync_df is not None and not three_sync_df.empty:
    # Count available plots
    plot_count = len(figs) if figs else 0
    
    # Check for XY plots
    has_xy = ('sonar_x' in three_sync_df.columns or 
              'dvl_x' in three_sync_df.columns or 
              'fft_x' in three_sync_df.columns)
    
    if has_xy:
        plot_count += 3  # XY trajectory + X comparison + Y comparison
    
    print(f"\nGenerated Plots: {plot_count}")
    if figs:
        for name in figs.keys():
            print(f"  - {name.replace('_', ' ').title()}")
    if has_xy:
        print(f"  - XY Trajectories")
        print(f"  - X Position Comparison")
        print(f"  - Y Position Comparison")
    
    print(f"\nSaved to: {PLOTS_DIR}")

print("\n✓ Notebook complete")


ANALYSIS SUMMARY

Target Bag: 2024-08-20_17-02-00

Data Availability:
  ✓ Sonar: 944 records
  ✓ DVL: 497 records
  ✓ FFT: 410 records

Synchronization Results:
  Two-system (Sonar+DVL): 452 records
  Multi-system: 973 records

Generated Plots: 5
  - Distance Comparison
  - Pitch Comparison
  - XY Trajectories
  - X Position Comparison
  - Y Position Comparison

Saved to: /Volumes/LaCie/SOLAQUA/exports/plots

✓ Notebook complete
